In [3]:
import sys,os
sys.path.append("/home/paco/Documents/site_similarity")

In [4]:
from utils.notebook_utils import load_level_data, create_weighted_nodes, create_graph, draw_graph

In [4]:
lvl_one = load_level_data(data_path='/home/paco/Documents/site_similarity/notebooks/clean_data_20200808.json', level=1)

Loaded 3489 nodes with records level <= 1 and child size:16981


In [5]:
lvl_one_nodes = create_weighted_nodes(lvl_one)

In [6]:
lvl_one_nodes[:10]

[('crooked.com', 'votesaveamerica.com', 23.7),
 ('crooked.com', 'art19.com', 6.6),
 ('crooked.com', 'promocodeportal.com', 5.0),
 ('crooked.com', 'mediamatters.org', 4.2),
 ('crooked.com', 'actblue.com', 4.2),
 ('deepleftfield.info', 'politicalflare.com', 1.9),
 ('deepleftfield.info', 'bluedotdaily.com', 1.2),
 ('antifascistnews.net', 'angrywhitemen.org', 8.2),
 ('antifascistnews.net', 'itsgoingdown.org', 5.3),
 ('antifascistnews.net', 'torchantifa.org', 5.0)]

In [7]:
[(k, v, w) for k, v, w in lvl_one_nodes if k == 'crooked.com']

[('crooked.com', 'votesaveamerica.com', 23.7),
 ('crooked.com', 'art19.com', 6.6),
 ('crooked.com', 'promocodeportal.com', 5.0),
 ('crooked.com', 'mediamatters.org', 4.2),
 ('crooked.com', 'actblue.com', 4.2)]

In [8]:
from stellargraph import StellarGraph
import pandas as pd


lvl_one_graph = StellarGraph(edges=pd.DataFrame(lvl_one_nodes, columns=['source', 'target', 'weight']))

In [9]:
print(lvl_one_graph.info())

StellarGraph: Undirected multigraph
 Nodes: 10161, Edges: 17010

 Node types:
  default: [10161]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [17010]
        Weights: range=[0.5, 98.7], mean=12.9548, std=13.1882
        Features: none


In [10]:
from stellargraph.data import BiasedRandomWalk

rw = BiasedRandomWalk(lvl_one_graph)

walks = rw.run(
    nodes=list(lvl_one_graph.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
    weighted=True,  # for weighted random walks
    seed=42,  # random seed fixed for reproducibility
)
print("Number of random walks: {}".format(len(walks)))

Number of random walks: 101610


In [11]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=64, window=5, min_count=0, sg=1, workers=2, iter=1)

model.save("node2vec_lvl_one_new_corpus_WEIGHTED_64.model")

In [6]:
# Load model

from gensim.models import Word2Vec

model = Word2Vec.load("node2vec_lvl_one_new_corpus_WEIGHTED_64.model")

In [7]:
model.wv['crooked.com'].shape

(64,)

In [8]:
from dataprep.load_annotated_data import apply_splits
from utils.notebook_utils import load_corpus
from modelling.baselines import eval_model
import pandas as pd

DATA = load_corpus('new_corpus_2020.csv')
SPLITS = apply_splits(DATA, 'modified_splits_new_corpus_2020.json')

print(SPLITS.keys())

train = pd.DataFrame(SPLITS['train-0'])

test = pd.DataFrame(SPLITS['test-0'])

dict_keys(['test-0', 'train-0', 'test-1', 'train-1', 'test-2', 'train-2', 'test-3', 'train-3', 'test-4', 'train-4'])


In [9]:
X_train = train.apply(lambda x: model.wv[x['source_url_processed']], axis=1)

y_train = train.apply(lambda x: x['fact'], axis=1)

In [10]:
X_test = test.apply(lambda x: model.wv[x['source_url_processed']], axis=1)

y_test = test.apply(lambda x: x['fact'], axis=1)

In [11]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

clf = LogisticRegressionCV(
    Cs=10, cv=5, scoring="accuracy", verbose=True, multi_class="ovr", max_iter=300, random_state=42
)
clf.fit(X_train.values.tolist(), y_train.values.tolist())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.8s finished


LogisticRegressionCV(cv=5, max_iter=300, multi_class='ovr', random_state=42,
                     scoring='accuracy', verbose=True)

In [12]:
y_pred = clf.predict(X_test.values.tolist())

accuracy_score(y_test, y_pred)

0.5988372093023255

In [18]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train.tolist(), y_train.tolist())

GradientBoostingClassifier(random_state=42)

In [19]:
tree_predict = clf.predict(X_test.tolist())
accuracy_score(y_test, tree_predict)

0.6337209302325582

In [20]:
from sklearn import svm

svm_clf = svm.SVC(decision_function_shape='ovo')

svm_clf.fit(X_train.tolist(), y_train.tolist())

SVC(decision_function_shape='ovo')

In [21]:
svm_predict = svm_clf.predict(X_test.tolist())
accuracy_score(y_test, svm_predict)

0.6744186046511628

In [22]:
lin_clf = svm.LinearSVC() # one-vs-rest
lin_clf.fit(X_train.tolist(), y_train.tolist())

LinearSVC()

In [23]:
lin_predict = lin_clf.predict(X_test.tolist())
accuracy_score(y_test, lin_predict)

0.6104651162790697